Libraries

In [ ]:
import os
import re
import yaml
import json
import glob
import time
from datetime import datetime
import logging
import tempfile
import subprocess
from typing import Dict, Optional, List, Tuple
from concurrent.futures import ProcessPoolExecutor, as_completed


import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype, is_categorical_dtype
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, datasets
import SimpleITK as sitk
from radiomics import featureextractor
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, mutual_info_classif, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, roc_curve, auc,
    classification_report, mean_squared_error, mean_absolute_error,
    r2_score, explained_variance_score, mean_absolute_percentage_error
)
from mAIstro_tools import (
    PyRadiomicsFeatureExtractionTool,
    EDAToolException,
    ExploratoryDataAnalysisTool,
    FeatureImportanceAnalysisTool,
    NNUNetTrainingTool,
    NNUNetInferenceTool,
    TotalSegmentatorTool,
    PyCaretClassificationTool,
    PyCaretInferenceTool,
    PyCaretRegressionInferenceTool,
    PyCaretRegressionTool,
    MedicalImageDataset,
    PyTorchResNetTrainingTool,
    PyTorchResNetInferenceTool,
    PyTorchVGG16InferenceTool,
    PyTorchVGG16TrainingTool,
    PyTorchInceptionV3InferenceTool,
    PyTorchInceptionV3TrainingTool
)
from smolagents import Tool, CodeAgent, LiteLLMModel, GradioUI, HfApiModel

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Tools Setup

In [ ]:
pyradiomics_feature_extraction_tool = PyRadiomicsFeatureExtractionTool()
eda_tool = ExploratoryDataAnalysisTool()
feature_selection_tool = FeatureImportanceAnalysisTool()
nnunet_training_tool = NNUNetTrainingTool()
nnunet_inference_tool = NNUNetInferenceTool()
totalsegmentator_tool = TotalSegmentatorTool()
pycaret_class_inference_tool = PyCaretInferenceTool()
pycaret_class_training_tool = PyCaretClassificationTool()
pycaret_regression_training_tool = PyCaretRegressionTool()
pycaret_regression_inference_tool = PyCaretRegressionInferenceTool()
pytorch_resnet_inference_tool = PyTorchResNetInferenceTool()
pytorch_resnet_training_tool = PyTorchResNetTrainingTool()
pytorch_vgg16_inference_tool = PyTorchVGG16InferenceTool()
pytorch_vgg16_training_tool = PyTorchVGG16TrainingTool()
pytorch_inceptionv3_inference_tool = PyTorchInceptionV3InferenceTool()
pytorch_inceptionv3_training_tool = PyTorchInceptionV3TrainingTool()

LLM Setup

Use models through API calls

In [ ]:
#gpt-4.1
model = LiteLLMModel(model_id="openai/gpt-4.1", api_key="YOUR_API_KEY_HERE")
#gpt-4o
#model = LiteLLMModel(model_id="openai/gpt-4o", api_key="YOUR_API_KEY_HERE")
#claude
#model = LiteLLMModel(model_id="anthropic/claude-3-7-sonnet-20250219", api_key="YOUR_API_KEY_HERE") 
#deepseek V3
#model = LiteLLMModel(model_id="deepseek/deepseek-chat", api_key="YOUR_API_KEY_HERE")
#deepseek R1
#model = LiteLLMModel(model_id="deepseek/deepseek-reasoner", api_key="YOUR_API_KEY_HERE")

#hf API
#Qwen
#model_id = "Qwen/QwQ-32B" 
#model = HfApiModel(model_id=model_id, token="YOUR_API_KEY_HERE")
#Llama 3.3
#model_id = "meta-llama/Llama-3.3-70B-Instruct" 
#model = HfApiModel(model_id=model_id, token="YOUR_API_KEY_HERE")

#Together AI API
#llama-4-Scout
#model = LiteLLMModel(model_id="together_ai/meta-llama/Llama-4-Scout-17B-16E-Instruct", api_key="YOUR_API_KEY_HERE")
#QwQ-32B
#model = LiteLLMModel(model_id="together_ai/Qwen/QwQ-32B", api_key="YOUR_API_KEY_HERE")
#Llama 3.3
#model = LiteLLMModel(model_id="together_ai/meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", api_key="YOUR_API_KEY_HERE")
#Llama 3.1 8b
#model = LiteLLMModel(model_id="together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-classifier", api_key="YOUR_API_KEY_HERE")
#Mistral 24b
#model = LiteLLMModel(model_id="together_ai/mistralai/Mistral-Small-24B-Instruct-2501", api_key="YOUR_API_KEY_HERE")
#Mistral 7b
#model = LiteLLMModel(model_id="together_ai/mistralai/Mistral-7B-Instruct-v0.2", api_key="YOUR_API_KEY_HERE")
#DeepSeek R1 Distill Qwen 14B
#model = LiteLLMModel(model_id="together_ai/deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", api_key="YOUR_API_KEY_HERE")


Agents Setup

In [ ]:
#Radiomic extraction Agent
radiomic_extraction_agent = CodeAgent(
    tools = [pyradiomics_feature_extraction_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['os', 'pandas'],
    name="radiomic_extraction_agent",
    description="Extracts radiomic features from medical images and saves them as CSV files",
    max_steps = 5
)
#EDA Agent
eda_agent = CodeAgent(
    tools = [eda_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['os', 'pandas'],
    name="eda_agent",
    description="Performs comprehensive exploratory data analysis",
    max_steps = 5
)
#Feature Importance Agent
feature_selection_agent = CodeAgent(
    tools = [feature_selection_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['os', 'pandas'],
    name="feature_importance_agent",
    description="Performs feature importance analysis and exports the most important features to CSV files",
    max_steps = 5
)
#nnUNet Agent
nnunet_agent = agent = CodeAgent(
    tools = [nnunet_training_tool, nnunet_inference_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['re', 'subprocess', 'os'],
    name="nnunet_agent",
    description="Uses the nnUNet framework for training and inference of medical image segmentation models",
    max_steps = 5
)
#TotalSegmentator Agent
totalsegmentator_agent = CodeAgent(
    tools = [totalsegmentator_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['os'],
    name="totalsegmentator_agent",
    description="Utilizes the TotalSegmentator framework to segment organs and tissues in medical imaging data",
    max_steps = 5
)
#PyCaret classification Agent (tabulated data)
pycaret_classification_agent = CodeAgent(
    tools = [pycaret_class_training_tool, pycaret_class_inference_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['pycaret', 'setup', 'compare_models', 'tune_model', 'blend_models', 
                    'pull', 'predict_model', 'save_model',
                    'plot_model', 'interpret_model', 'cuml', 'pandas'],
    name="pycaret_classification_agent",
    description="Builds and deploys classification models using the PyCaret framework on tabular data inputs",
    max_steps = 5
)
#PyCaret regression Agent (tabulated data)
pycaret_regression_agent = CodeAgent(
    tools = [pycaret_regression_training_tool, pycaret_regression_inference_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['pycaret', 'setup', 'compare_models', 'tune_model', 'blend_models', 
                    'pull', 'predict_model', 'save_model',
                    'plot_model', 'interpret_model', 'cuml', 'pandas'],
    name="pycaret_regression_agent",
    description="Builds and deploys regression models using the PyCaret framework on tabular data inputs",
    max_steps = 5
)
#Image classification Agent
image_classification_agent = CodeAgent(
    tools = [pytorch_resnet_training_tool, pytorch_resnet_inference_tool, 
             pytorch_inceptionv3_training_tool, pytorch_inceptionv3_inference_tool,
             pytorch_vgg16_training_tool, pytorch_vgg16_inference_tool],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['os'],
    name="image_classification_agent",
    description="Builds and deploys ResNet, Inceptionv3 and VGG16 image classification models",
    max_steps = 5
)

#Master Agent
master_agent = CodeAgent(
    tools = [],
    managed_agents=[radiomic_extraction_agent, eda_agent, feature_selection_agent,
                    nnunet_agent, totalsegmentator_agent, pycaret_classification_agent,
                    pycaret_regression_agent, image_classification_agent],
    model=model,
    add_base_tools=True,
    additional_authorized_imports=['os'],
    name="master_agent",
    max_steps = 15
)

In [ ]:
master_agent.visualize()

Prompts

Radiomic Feature Extraction from CT

In [ ]:
#Test: Generic query
rfe_ct_prompt_1 = """
Perform a comprehensive radiomic feature extraction for the CT scans in: "/path/to/ct/images".
The corresponding masks are here: "/path/to/ct/labels".
Save the results here: "/path/to/output_directory".
"""

master_agent.run(rfe_ct_prompt_1)

In [ ]:
#Test: Ask for specific radiomic freatures and filters.
rfe_ct_prompt_2 = """
Extract shape and first order radiomic features for the CT scans in: "/path/to/ct/images".
The respective masks are here: "/path/to/ct/labels".
Save the results here: "/path/to/output_directory". 
Use the followng filters: Exponential, Gradient, LBP2D.
"""

master_agent.run(rfe_ct_prompt_2) 

Radiomic Feature Extraction from MRI

In [ ]:
#Test: Generic query for pre contrast MRI images (MAMAMIA dataset)
rfe_mri_prompt_1 = """
Perform a comprehensive radiomic feature extraction for the MR scans in: "/path/to/mri/mama_mia/images_pre_contrast".
The respective masks are here: "/path/to//mri/mama_mia/labels". 
Save the results here: "/path/to/output_directory".
"""
master_agent.run(rfe_mri_prompt_1) 

In [ ]:
#Test: Extraction of radiomic features from mpMRI images (BraTS21 dataset) with multiple labels.
#Asking specific features and filters

rfe_mri_prompt_2 = """
Extract shape and glrlm and ngtdm radiomic features for the MR scans in: "/path/to/mri/brats21/images".
The corresponding masks are here: "/path/to/mri/brats21/labels". 
Use the followng filters: Exponential, Gradient, SquareRoot.
Save the results here: "/path/to/output_directory".
"""
master_agent.run(rfe_mri_prompt_2) 

Exploratory Data Analysis

In [ ]:
#Prompt for breast wisconsin dataset
eda_prompt_1 = """
Perform comprehensive exploratory data analysis for the file: "/path/to/breast_cancer_wisconsin_diagnosis_datasetdata.csv". 
Save the output here: "/path/to/output_directory".
"""

#Prompt for predict_diabetes dataset
eda_prompt_2 ="""
Perform comprehensive EDA for the file: /path/to/predict_diabetes.csv
Save the output here: "/path/to/output_directory"
"""

#Prompt for heart disease dataset
eda_prompt_3 = """
Perform comprehensive EDA for the file: /path/to/heart_disease_classification.csv. 
Save the results here: "/path/to/output_directory".
"""

#Prompt for heart failure dataset
eda_prompt_4 = """
Perform EDA for the file: /path/to/heart_failure_clinical_records_dataset.csv 
Save the results here: "/path/to/output_directory".
"""

#Prompt for life expectancy dataset
eda_prompt_5 = """
Perform comprehensive EDA for the file: /path/to/Life-Expectancy-Data.csv 
Save the results here: "/path/to/output_directory".
"""

master_agent.run(eda_prompt_1) 

Feature Importance Analysis and Feature Extraction

In [ ]:
#Prompt for breast wisconsin dataset
fia_prompt_1 = """
Perform feature importance analysis for the file: "/path/to/breast_cancer_wisconsin_diagnosis_datasetdata.csv".
Save three csv files with the top 5, 10 and 20 features here: "/path/to/output_directory".
Targert column is "diagnosis".
"""

#Prompt for predict_diabetes dataset !Asking to export more features than the original file has.
fia_prompt_2 = """
        Perform feature importance analysis for the file: "/path/to/predict_diabetes.csv".
        Save three csv files with the top 5, 10 and 20 features here: "/path/to/output_directory".
        Targert column is "Outcome".
        """

#Prompt for heart disease dataset
fia_prompt_3 = """
        Perform feature importance analysis for the file: "/path/to/heart_disease_classification.csv".
        Save three csv files with the top 5, 10 features here: "/path/to/output_directory".
        Targert column is "target".
        """

#Prompt for heart failure dataset
fia_prompt_4 = """
        Perform feature importance analysis for the file: "/path/to/heart_failure_clinical_records_dataset.csv".
        Save three csv files with the top 8 features here: "/path/to/output_directory".
        Targert column is "DEATH_EVENT".
        """

#Prompt for life expectancy dataset
fia_prompt_5 = """
        Perform feature importance analysis for the file: /path/to/Life-Expectancy-Data-Updated.csv
        Save two csv files with the top 10 and 15 features here: "/path/to/output_directory"
        Targert column is "Life_expectancy". Create plots.
        """

master_agent.run(fia_prompt_1) 

nnUNet framework - Train and Inference

In [ ]:
#Prompt for training segmentation UNet (Brats21 dataset)
nnunet_prompt_1 = 'Train a segmentation 3d full res for the dataset in: /nnUNet_raw/Dataset135_Brats21. For fold all'

#Prompt for training segmentation UNet (Kits23 dataset)
nnunet_prompt_1 = 'Train a segmentation 3d full res for the dataset in: /nnUNet_raw/Dataset140_Kits23. For fold all'

#Prompt for inference mpMRI (Brats21 dataset)
nnunet_prompt_2 ="""
Using the nnUNet dataset 135, 3d full res fold_all model, segment the scans in: /inference_nnunet/brats21_validation. 
Output folder: /path/to/ouput_directory
"""

#Prompt for inference mpMRI (Kits23 dataset)
nnunet_prompt_2 ="""
Using the nnUNet dataset 140, 3d full res fold_all model, segment the scans in: /inference_nnunet/kits23_validation. 
Output folder: /path/to/ouput_directory
"""

master_agent.run(nnunet_prompt_2) 

TotalSegmentator Inference

In [ ]:
#Prompt 1 segmenting only one organ (CT)
totalsegmentator_prompt_1 = """
Use TotalSegmentator with the total task to segment only the spleen in the CT scan located at /path/to/ct_input 
Save the mask in this directory: /path/to/ouput_directory
"""

#Prompt 2 segmenting three organs (CT)
totalsegmentator_prompt_2 = """
Use TotalSegmentator with the total task to segment only the liver, stomach and kidneys in the CT scan found at /path/to/ct_input  
Save the mask here /path/to/ouput_directory
"""

#Prompt 3 segmenting all available organs (CT)
totalsegmentator_prompt_3 = """
Use TotalSegmentator with the total task to segment all available organs in the CT scan located at "/path/to/ct_input".  
Save the mask in this folder: /path/to/ouput_directory
"""

#Prompt 4 segmenting all available organs (MR)
totalsegmentator_prompt_4 = """
Use TotalSegmentator with the total_mr task to segment all available organs in the MRI scan found at "/path/to/ct_input"
Save the mask in this folder: "/path/to/ouput_directory".
"""

master_agent.run(totalsegmentator_prompt_4) 

Training prompts Classification Model (Tabulated data) 

In [ ]:
#Prompt for classification model development Breast wisconsin
tct_prompt_1 = """
Train a classification model using the tabulated data: /path/to/breast_cancer_wisconsin_diagnosis_datasetdata.csv.
Target column: "diagnosis". Exclude lightgbm classifier. Set normalization and transormation to False.
Save the results here: "/path/to/ouput_directory".
"""

#Prompt for classification model development Predict diabetes
tct_prompt_2 = """
Train a classification model using the file: /path/to/predict_diabetes.csv. 
Target column: "Outcome". Exclude lightgbm classifier.
Save the results here: "/path/to/ouput_directory".
"""

#Prompt for classification model development Predict Heart Disease
tct_prompt_3 = """
Train a classification model using the file: /path/to/heart_disease_classification.csv
Target column: "target". Exclude lightgbm classifier.
Save the results here: "/path/to/ouput_directory".
"""

#Prompt for classification model development Predict Heart Failure
tct_prompt_4 = """
Train a classification model using the file: /path/to/heart_failure_clinical_records_dataset.csv
Target column: "DEATH_EVENT". Exclude lightgbm, dummy and catboost classifier.
Save the results here: "/path/to/ouput_directory"
"""

master_agent.run(tct_prompt_1) 

Inference prompts Classification Model (Tabulated data) 

In [ ]:
#Prompt for classification inference Breast wisconsin tuned model 1
ict_prompt_1 = """
Use the classification model: /path/to/breast_cancer_wisconsin/models/tuned_model_1/tuned_model_1.pkl 
Make predictions using the predictors in the file: /path/to/inferer_results/independent_eval_cohort.csv 
The ground truth values are in the column: "diagnosis"
Output directory: "/path/to/ouput_directory"
"""

#Prompt for classification inference Predict diabetes tuned model 3
ict_prompt_2 = """
Use the classification model: /path/to/predict_diabetes/models/tuned_model_3/tuned_model_3.pkl 
Make predictions using the predictors in the file: /path/to/predict_diabetes/test_set.csv 
The ground truth values are in the column: "Outcome"
Output directory: "/path/to/ouput_directory"
Deploy the proper agent for this task.
"""

#Prompt for classification inference Predict diabetes blended model
ict_prompt_3 = """
Use the classification model: /path/to/predict_diabetes/models/blended_model/blended_model.pkl
Make predictions using the predictors in the file: /path/to/predict_diabetes/test_set.csv 
The ground truth values are in the column: "Outcome"
Output directory: "/path/to/ouput_directory"
"""

#Prompt for classification inference Predict heart disease tuned model 3
ict_prompt_4 = """
Use the classification model: /path/to/heart_disease/models/tuned_model_3/tuned_model_3.pkl
Make predictions using the features in the file: /path/to/heart_disease/independent_eval_cohort.csv
The ground truth values are in the column: "target"
Output directory: "/path/to/ouput_directory"
"""

#Prompt for classification inference Predict heart failure tuned model 2
ict_prompt_5 = """
Use the classification model: /path/to/heart_failure/models/tuned_model_2/tuned_model_2.pkl
Make predictions using the predictors in the file: /path/to//heart_failure/independent_eval_cohort.csv
The ground truth values are in the column: "DEATH_EVENT"
Output directory: "/path/to/ouput_directory"
"""

master_agent.run(ict_prompt_5) 

Training prompts Regression Model (Tabulated data) 

In [ ]:
#Prompt for regression model development Life expectancy dataset
trt_prompt_1 = """
Train a regression model using the file: /path/to/Life-Expectancy-Data.csv 
Exclude "lightgbm".
Target column: "Life_expectancy".
Save the results here: "/path/to/ouput_directory"
"""

master_agent.run(trt_prompt_1) 

Inference prompts Regression Model (Tabulated data)

In [ ]:
#Prompt for regression inference Life expectancy (providing the gt for comparison)
irt_prompt_1 = """
Use the regression model: /path/to/life_expectancy/models/tuned_model_1/tuned_model_1.pkl
Make predictions using the predictors in the file: /path/to/life_expectancy/independent_eval_cohort.csv
The ground truth values are in the column: "Life_expectancy"
Output directory: /path/to/ouput_directory
Deploy the proper agent and tool for this task
"""

#Prompt for regression inference Life expectancy (without gt)
irt_prompt_2 = """
Use the regression model: /path/to/life_expectancy/models/tuned_model_1/tuned_model_1.pkl
Make predictions using the predictors in the file: /path/to/life_expectancy/no_gt_independent_eval_cohort.csv
Output directory: /path/to/ouput_directory
"""

master_agent.run(irt_prompt_1) 

Training Prompts - Image Classification

In [ ]:
#Prompt for classification resnet18 model development for pneumoniamnist_28 dataset
ict_prompt_1 = """
Deploy the appropriate agent and tool to train a classification resnet18 model. 
The train dataset directory: "/path/to/pneumoniamnist_28/dataset_pneumoniamnist_28/train",  
the validation dataset directory: "/path/to/pneumoniamnist_28/dataset_pneumoniamnist_28/val",
the test dataset directory: "/path/to/pneumoniamnist_28/dataset_pneumoniamnist_28/test", 
Number of classes 2. 
Use a batch size of 64. Number of epochs: 60
Output directory: "/path/to/ouput_directory" 
"""

#Prompt for classification resnet34 model development for pathmnist_64 dataset 
ict_prompt_2 = """
Train a classification resnet34 model. 
The train, validation and test datasets: /path/to/pathmnist_64/dataset_pathmnist_64. 
Number of classes 9. 
Use a batch size of 32. Set patience to 10 and number of epochs to 50.
Output directory: /path/to/ouput_directory 
"""

#Prompt for classification resnet50 model development for breastmnist_128 dataset
ict_prompt_3 = """
Train a classification resnet50 model. 
The train, val and test data are available here: "/path/to/breastmnist_128/dataset_breastmnist_128/".
Number of classes 2. 
Train for 50 epochs. Do not use early stopping. 
Output folder: "/path/to/ouput_directory"
"""

##Prompt for classification resnet101 model development for dermamnist_224 dataset
ict_prompt_4 = """
Train a classification resnet101 model. 
The train, val and test data are available here: "/path/to/dermamnist_224/dataset_dermamnist_224".
Number of classes 7. Train for 200 epochs. Set patience for early stopping to 10. 
Output folder: "/path/to/ouput_directory"
"""

##Prompt for classification resnet152 model development for organamnist_28 dataset
ict_prompt_5 = """
Train a classification resnet152 model. 
The train, val and test data are available here: "/path/to/organamnist_28/dataset_organamnist_28/".
Number of classes 11. Set patience to 5.
Output folder: "/path/to/ouput_directory"
"""

##Prompt for classification vgg16 model development for octmnist_28 dataset
ict_prompt_6 = """
Train a classification vgg16 model. 
The train, val and test data are available here: "/path/to/octmnist_28/dataset_octmnist_28/".
Number of classes 4. Do not use pretrained weights.
Output folder: "/path/to/ouput_directory"
"""

##Prompt for classification InceptionV3 model development for bloodmnist_128 dataset
ict_prompt_7 = """
Train a classification InceptionV3 model. 
The train, val and test data are available here: "/path/to/bloodmnist_128/dataset_bloodmnist_128".
Number of classes 8. Use pretrained weights and a batch size of 64. Train for 150 epochs.
Output folder: "/path/to/ouput_directory"
"""

master_agent.run(ict_prompt_5) 

Inference Prompts - Image Classification

In [ ]:
#Prompt for inference with resnet18 model - pneumoniamnist_28 dataset
ici_prompt_1 = """
Use the resnet-18 model available here: "/path/to/pneumoniamnist_28/output_master_agent/resnet18_ict_prompt_1/best_model.pt", 
to classify the images in this folder: "/path/to/pneumoniamnist_28/dataset_pneumoniamnist_28/test".
The number of classes is 2. 
The ground truth labels for the evaluation are availabe here:"/path/to/pneumoniamnist_28/inference/gt_test_labels.csv".
Save the evaluation output in this directory: "/path/to/ouput_directory". 
Deploy the appropriate agent and tool for this task.
"""

#Prompt for inference with resnet34 model - pathmnist_64 dataset
ici_prompt_2 = """
Use the resnet34 model available here: "/path/to/pathmnist_64/output_master_agent/resnet34_ict_prompt_2/best_model.pt", 
to classify the images in this folder: "/path/to/pathmnist_64/dataset_pathmnist_64/test".
The number of classes is 9. 
The ground truth labels for the evaluation are availabe here:"/path/to/pathmnist_64/inference/gt_test_labels.csv".
Save the evaluation output in this directory: "/path/to/ouput_directory".
"""

#Prompt for inference with resnet50 model - breastmnist_128 dataset
ici_prompt_3 = """
Use the resnet50 model available here: "/path/to/breastmnist_128/output_master_agent/resnet50_ict_prompt_3/best_model.pt",
to classify the images in this folder: "/path/to/breastmnist_128/dataset_breastmnist_128/test".
The number of classes is 2. 
The ground truth labels for the evaluation are availabe here: "/path/to/breastmnist_128/inference/gt_test_labels.csv".
Output directory: "/path/to/ouput_directory".
"""

##Prompt for inference with resnet101 model - dermamnist_224 dataset
ici_prompt_4 = """
Use the resnet101 model available here: "/path/to/dermamnist_224/output_master_agent/resnet101_ict_prompt_4/best_model.pt",
to classify the images in this folder: "/path/to/dermamnist_224/dataset_dermamnist_224/test".
Number of classes 7. 
Output folder: "/path/to/ouput_directory".
"""

##Prompt for inference with resnet152 model - organamnist_28 dataset
ici_prompt_5 = """
Use the resnet152 model available here: "/path/to/dermamnist_224/output_master_agent/resnet101_ict_prompt_4/best_model.pt",
to classify the images in this folder: "/path/to/dermamnist_224/inference_test/images".
Number of classes 11. 
Output folder: "/path/to/ouput_directory".
"""

##Prompt for inference with vgg16 model - octmnist_28 dataset
ici_prompt_6 = """
Use the vgg16 model available here: "/path/to/octmnist_28/output_master_agent/vgg16_ict_prompt_6/best_model.pt",
to classify the images in this folder: "/path/to/octmnist_28/dataset_octmnist_28/test".
Number of classes 4. The ground truth labels are availabe in this directory: "/path/to/octmnist_28/inference".
Output folder: "/path/to/ouput_directory".
"""

##Prompt for inference with InceptionV3 model - bloodmnist_128 dataset
ici_prompt_7 = """
Use the InceptionV3 model available here: "/path/to/bloodmnist_128/output_master_agent/InceptionV3_ict_prompt_7/best_model.pt",
to classify the images in this folder: "/path/to/bloodmnist_128/dataset_bloodmnist_128/test".
Number of classes 8. The ground truth labels are availabe in this directory: "/path/to/bloodmnist_128/inference/gt_test_labels.csv".
Output folder: "/path/to/ouput_directory".
"""

master_agent.run(ici_prompt_1)

In [ ]:
multi_task_prompt_1 = """
Use TotalSegmentator with the total task to segment only the spleen from the CT scans (one at a time) available here: "/path/to/test_multi_tasking/multi_task_prompt_1/ct_scans". 
Save the mask files to "/path/to/test_multi_tasking/multi_task_prompt_1/masks".
Rename each mask file to have the exact same filename with the corresponding CT scan.
Use the CT scans and the corresponding spleen masks to extract radiomic features. 
Save the resulting radiomic features CSV file in here: "/path/to/ouput_directory".
Use the saved csv file with the radiomic features to perform Exploratory Data Analysis, without producing plots, on the extracted features and save all EDA results in this directory: "/path/to/ouput_directory".
Deploy the appropriate agent for each task.
"""

multi_task_prompt_2 = """
Using the nnUNet dataset 135, 3d full res fold_all model, segment the mpMRI scans in: "/path/to/test_multi_tasking/multi_task_prompt_2/mpMRI_scans". 
Save the mask files to: "/path/to/test_multi_tasking/multi_task_prompt_2/masks"
Use the MRI scans and the corresponding masks to extract radiomic features for each label in the mask files. Use the followng filters: Exponential, Original, Wavelet.
Extract radiomic features only from the T1 MRI scans. For example, the mask name for the first case will be 'BraTS2021_00000.nii.gz', the corresponding T1 image name will be 'BraTS2021_00000_0000.nii.gz'.
Save the resulting radiomic features CSV files in this folder: "/path/to/ouput_directory".
Use the saved csv files with the radiomic features to perform Exploratory Data Analysis. Save EDA results for each label in subfolders, in this directory:"/path/to/ouput_directory".
"""

multi_task_prompt_3 = """
Perform a feature importance analysis for the file: "/path/to/test_multi_tasking/multi_task_prompt_3/mamamia_features/dataset_mamamia.csv".
Save three csv files with the top 20, 50 and 100 features here: "/path/to/ouput_directory".
Targert column is "pcr".
Then use the csv file with the top 20 features to train a classifier. Save the output of the training process in this directory: "/path/to/ouput_directory".
"""

multi_task_prompt_4 = """
Perform exploratory data analysis for the file: "/path/to/breast_cancer_wisconsin_diagnosis_dataset.csv".
Save eda results and a csv files with the top 10 features here: "/path/to/ouput_directory".
Targert column is "diagnosis".
Then use the csv file with the top 10 features to train a classifier. Save the output of the training process in this directory: "/path/to/ouput_directory".
"""

multi_task_prompt_5 = """
Train an InceptionV3 classification model using data available in the following directory: "/path/to/pneumoniamnist_128/dataset_pneumoniamnist_128".
The number of classes is 2. Train for 100 epochs. Set patience to 10.
Then, use the trained model to classify images in this directory: "/path/to/pneumoniamnist_128/inference/test_data".
The ground truth labels for the evaluation are availabe here: "/path/to/pneumoniamnist_128/inference/gt_test_labels.csv".
Save the trained model and inference results in this directory: "/path/to/ouput_directory".
"""

master_agent.run(multi_task_prompt_5)

For Interactive communication with the master agent run the following cell

In [ ]:
GradioUI(master_agent).launch()